In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature
import copy
from constraint_relax import search_mfs
from item_extrator import data_path, dummy_data_path
from similarity_calculator import compute_similarity

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
# read experiment constraints data
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        if constraint != None:
            constraint_list.append(constraint)
    cl_set = set(constraint_list)
    problem.constraint_list = set(constraint_list)
    problem_list.append(problem)

In [4]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

sol_list = []
threshold_list = [0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = []
    p1 = copy.deepcopy(problem) # original constraint
    prob_sol_list.append(p1.constraint_list)
    prob_sol_list.append(get_df_len(p1.solve()))

    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), dummy_data_path)
        p.relax(mfs, threshold)
        prob_sol_list.append(p.constraint_list)
        prob_items = p.solve()
        prob_sol_list.append(get_df_len(prob_items))
        if not prob_items is None:
            prob_sol_list.append((compute_similarity(p.constraint_list, prob_items))['similarity'].mean())
        else:
            prob_sol_list.append(0)
    sol_list.append(prob_sol_list)
result = pd.DataFrame(sol_list)

In [5]:
result

,0,1,2,3,4,5,6,7,8,9,10
0,"{price: 2699.0: 10: Nature.LESS, camera: True:...",14,"{price: 2699.0: 10: Nature.LESS, os: 10: 4: Na...",15,0.720075,"{price: 2699.0: 10: Nature.LESS, camera: : 1: ...",15,0.720075,"{os: 10: 4: Nature.EQUAL, price: 2699.0: 10: N...",15,0.720075
1,"{weight: 2.0: 9: Nature.LESS, brand: asus|dell...",0,"{os: os: 8: Nature.EQUAL, weight: 2.0: 9: Natu...",0,0.000000,"{price: 3821.04: 6: Nature.LESS, brand: asus|d...",0,0.000000,"{weight: 2.0: 9: Nature.LESS, os: os: 8: Natur...",0,0.000000
2,"{ram: 16.0: 9: Nature.MORE, storage: 128.0: 3:...",0,"{ram: 16.0: 9: Nature.MORE, weight: 1.44: 9: N...",0,0.000000,"{os: 11: 4: Nature.EQUAL, ram: 16.0: 9: Nature...",0,0.000000,"{weight: 1.44: 9: Nature.LESS, ram: 16.0: 9: N...",0,0.000000
3,"{camera: True: 9: Nature.EQUAL, storage: 1024....",0,"{storage: 1024.0: 7: Nature.MORE, camera: True...",0,0.000000,"{brand: : 2: Nature.EQUAL, storage: 1024.0: 7:...",0,0.000000,"{storage: 1024.0: 7: Nature.MORE, price: 4997....",0,0.000000
4,"{cpu: ryzen 5|i7: 4: Nature.EQUAL, ram: 16.0: ...",5,"{os: 11: 4: Nature.EQUAL, cpu: ryzen 5|i7: 4: ...",6,0.639378,"{ram: 16.0: 7: Nature.MORE, cpu: ryzen 5|i7: 4...",6,0.624030,"{cpu: ryzen 5|i7: 4: Nature.EQUAL, ram: 16.0: ...",6,0.624030
...,...,...,...,...,...,...,...,...,...,...,...
495,"{brand: msi|acer|hp|huawei: 1: Nature.EQUAL, s...",0,"{price: 4097.0: 3: Nature.LESS, os: os: 9: Nat...",0,0.000000,"{ram: 4.0: 6: Nature.MORE, brand: : 1: Nature....",0,0.000000,"{brand: : 1: Nature.EQUAL, price: 4097.0: 3: N...",0,0.000000
496,"{ram: 4.0: 2: Nature.MORE, storage: 1024.0: 4:...",3,"{weight: 1.47: 7: Nature.LESS, ram: 2: 2: Natu...",3,0.676014,"{weight: 1.47: 7: Nature.LESS, ram: : 2: Natur...",3,0.687003,"{storage: 1024.0: 4: Nature.MORE, ram: : 2: Na...",3,0.687003
497,"{weight: 1.44: 1: Nature.LESS, os: 11: 2: Natu...",9,"{camera: : 1: Nature.EQUAL, storage: 256.0: 6:...",10,0.407405,"{os: 11: 2: Nature.EQUAL, camera: : 1: Nature....",10,0.407405,"{os: 11: 2: Nature.EQUAL, weight: : 1: Nature....",30,0.435484
498,"{cpu: i5|ryzen 3: 2: Nature.EQUAL, brand: leno...",0,"{storage: 1024.0: 5: Nature.MORE, cpu: ryzen 5...",0,0.000000,"{brand: lenovo|huawei: 8: Nature.EQUAL, cpu: :...",1,0.920000,"{brand: lenovo|huawei: 8: Nature.EQUAL, os: 10...",1,0.920000


In [6]:
pd.value_counts(result.iloc[:, 1] == 0)

True     285
False    215
Name: 1, dtype: int64

In [7]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 3]) & (result.iloc[:, 3] <= result.iloc[:, 6]) & (result.iloc[:, 6] <= result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     475
False     25
dtype: int64


In [8]:
valid = ((result.iloc[:, 1] < result.iloc[:, 3]) & (result.iloc[:, 3] < result.iloc[:, 6]) & (result.iloc[:, 6] < result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

False    496
True       4
dtype: int64


In [9]:
result[valid]

,0,1,2,3,4,5,6,7,8,9,10
109,"{cpu: i3|ryzen 3: 2: Nature.EQUAL, camera: Tru...",2,"{brand: hp|acer: 8: Nature.EQUAL, weight: 1.70...",3,0.825744,"{weight: : 2: Nature.LESS, ram: 8.0: 4: Nature...",4,0.791349,"{camera: True: 9: Nature.EQUAL, price: 3528.78...",8,0.803675
229,"{camera: True: 3: Nature.EQUAL, price: 4706.33...",12,"{camera: True: 3: Nature.EQUAL, storage: 512.0...",14,0.658962,"{camera: True: 3: Nature.EQUAL, weight: 2.47: ...",16,0.633422,"{os: 10: 6: Nature.EQUAL, weight: 2.47: 4: Nat...",47,0.619379
387,"{weight: 1.47: 5: Nature.LESS, ram: 16.0: 6: N...",6,"{weight: 1.5715: 5: Nature.LESS, ram: 16.0: 6:...",8,0.677725,"{brand: : 5: Nature.EQUAL, weight: 1.47: 5: Na...",13,0.380850,"{ram: 16.0: 6: Nature.MORE, weight: : 5: Natur...",15,0.473214
414,"{price: 3552.92: 9: Nature.LESS, camera: True:...",41,"{price: 3552.92: 9: Nature.LESS, camera: : 5: ...",45,0.626865,"{price: 3552.92: 9: Nature.LESS, os: 10|11|os:...",54,0.890023,"{os: : 5: Nature.EQUAL, price: 3552.92: 9: Nat...",55,0.626865
